In [1]:
# import run
# import nna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend("sox_io")

import numpy as np

from nna.exp import modelArchs,runutils

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import nna
import pandas as pd

from nna import visutils
from nna import dataimport
from nna import fileUtils

In [3]:
from torch.utils.data import Dataset

class audioDatasetInfer(Dataset):

    def __init__(self, X, y=None, transform=None):
        '''
    Args:

    '''
        self.X = X
        self.y = y
        #         self.landmarks_frame = pd.read_csv(csv_file)
        #         self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.y is None:
            sample =  self.X[idx], torch.zeros((2))
        else:
            sample = self.X[idx], self.y[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self, maxMelLen, sampling_rate,device):
        # sr = 44100 etc
        self.maxMelLen = maxMelLen
        self.sampling_rate = sampling_rate
        self.device=device
        #https://github.com/PCerles/audio/blob/3803d0b27a4e13efa760227ef6c71d0f3753aa98/test/test_transforms.py#L262
        #librosa defaults
        n_fft = 2048
        hop_length = 512
        power = 2.0
        n_mels = 128
        n_mfcc = 40
        # htk is false in librosa, no setting in torchaudio -?
        # norm is 1 in librosa, no setting in torchaudio -?
        self.melspect_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sampling_rate, window_fn=torch.hann_window,
                                                                  hop_length=hop_length, n_mels=n_mels, n_fft=n_fft).to(device)

    
        self.db_transform = torchaudio.transforms.AmplitudeToDB("power", 80.).to(device)
    def __call__(self, sample):
        x, y = sample
        x=x.to(self.device)
        mel = self.melspect_transform(x.reshape(-1))
        an_x = self.db_transform(mel)
        #librosa version
#         mel = librosa.feature.melspectrogram(y=x.reshape(-1),
#                                              sr=self.sampling_rate)
#         an_x = librosa.power_to_db(mel, ref=np.max)
#         an_x = an_x.astype("float32")
#         y = y.astype('float32')
#         print(an_x.shape)
        an_x = an_x[:, :self.maxMelLen]
        # 2-d conv
#         x = an_x.reshape(1, *an_x.shape[:])
        # 1-d conv
        x = an_x.reshape(1, an_x.shape[0]*an_x.shape[1])

        
        return x,y


In [4]:

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
  Utility function for computing output of convolutions
  takes a tuple of (h,w) and returns a tuple of (h,w)
  """
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor(((h_w[0] + (2 * pad) - (dilation *
                                      (kernel_size[0] - 1)) - 1) / stride) + 1)
    w = floor(((h_w[1] + (2 * pad) - (dilation *
                                      (kernel_size[1] - 1)) - 1) / stride) + 1)
    return h, w

# mel.shape,an_x.shape,X_train.shape
class testModel(nn.Module):
    '''A simple model for testing by overfitting.
    '''
    def __init__(self, out_channels, h_w, kernel_size, FLAT=False,output_shape=(10,)):
        # h_w: height will be always one since we use 1d convolution 
        super(testModel, self).__init__()
        self.out_channels = out_channels
        #### CONV
        self.conv1 = nn.Conv1d(in_channels=1, # depth of image == depth of filters
                               out_channels=self.out_channels, # number of filters 
                               kernel_size=kernel_size, # size of the filters/kernels
                               padding=1)

        self.conv1_shape = conv_output_shape(h_w, kernel_size=kernel_size, stride=1, pad=1, dilation=1)
        # conv is 1d
        self.conv1_shape = (1,self.conv1_shape[1])
        
        self.fc1 = nn.Linear(self.out_channels * self.conv1_shape[0] *self.conv1_shape[1], 75)  # 100

        self.fc2 = nn.Linear(75,output_shape[0])
        
        
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

        
    def forward(self, x):
#         x = x.reshape(1,)
#         print(x.shape) #  50,1,108800 (850*128)
        x = F.relu(self.conv1(x))
#         x = self.pool(x)
        # x = self.drop(x)
#         print(x.shape)# 58, 2, 108801
#         print(self.conv1_shape)
#         print(x.shape)
        x = x.view(-1, self.out_channels * self.conv1_shape[0] *self.conv1_shape[1])
        # batch_norm is missing
        x = F.relu((self.fc1(x)))
        x = (self.fc2(x))

#         x = self.drop(x)

#         x = self.fc4(x)
#         x = torch.sigmoid(x)
#                 x = F.log_softmax(x,dim=1)
        return x



In [5]:
expected_len = 10

def repeat_data(data):
    sr = 48000
    left_over=(data.shape[0])%(expected_len*sr)
    if left_over!=0:
        tile_reps = ((expected_len*sr)/(left_over)+1)
        # tile_reps
        repeated_data = np.tile(data[-left_over:],int(tile_reps))
        repeated_data=repeated_data[:expected_len*sr]
        return np.concatenate([data[:-left_over],repeated_data])
    else:
        return data



In [6]:
device="cuda:1"
CATEGORY_COUNT=2
# '1.1.10','1.1.7'
maxMelLen = 938
ToTensor_ins = ToTensor(maxMelLen,48000,device)

transformCompose = transforms.Compose([
    ToTensor_ins,
])

h_w = [128, 938]
kernel_size = (5, 5)


output_shape=(CATEGORY_COUNT,)



In [7]:
model_path="/home/enis/projects/nna/src/nna/exp/megan/run-1/wandb/run-20210114_155758-x6ltobzt/files/best_model_10_ROC_AUC=0.8853.pt"

model_saved = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
model_saved.load_state_dict(torch.load(model_path))
model_saved.eval().to(device)

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(25,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=240084, out_features=75, bias=True)
  (fc2): Linear(in_features=75, out_features=2, bias=True)
)

In [8]:
# #fake data
# sample_count=2000
# X = np.empty((sample_count*CATEGORY_COUNT,480000),dtype=np.float32)
# # y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
# y_true=[]
# for i in range(CATEGORY_COUNT):
#     y_true.extend([i]*sample_count)
# n_values = np.max(y_true) + 1
# y_true = np.eye(n_values)[y_true]

# X=torch.from_numpy(X).float()
# y_true=torch.from_numpy(y_true).float().to(device)

In [9]:
# X=torch.from_numpy(X).float()
# y_true=torch.from_numpy(y_true).float().to(device)

In [10]:
# %%time
# dataset = {'predict' : audioDatasetInfer(X, y_true, transform=transformCompose)}
# dataloader = {'predict': torch.utils.data.DataLoader(dataset['predict'], shuffle=False,batch_size=128)}

# outputs=[]
# for inputs, labels in dataloader['predict']:
#     inputs = inputs.float().to(device)
#     output = model_saved(inputs)
#     output = output.to("cpu")
#     index = output.data.numpy()
#     outputs.append(index)


In [11]:

file_properties_df=pd.read_pickle("/home/enis/projects/nna/data/realdata_v2No_stinchcomb.pkl")


In [12]:
region_location = tuple(sorted(set(zip(file_properties_df.region.values,file_properties_df.locationId.values))))

In [13]:
# file_properties_df[file_properties_df.region!='ivvavik'].iloc[0].name

In [14]:
# !ls /tank/data/nna/real/prudhoe/15/2019/S4A10283_20190531_164602.flac

In [15]:
# file_path = "/tank/data/nna/real/prudhoe/15/2019/S4A10283_20190531_164602.flac"
# dtype = ""
# sound_array, sr = nna.clippingutils.load_audio(file_path,
#                                                dtype=np.int16,
#                                                backend='pydub')

In [16]:
# sound_array.shape,sr
# region_location[0]


In [17]:
# 'anwr' only
region_location = tuple(sorted(set(zip(file_properties_df.region.values,file_properties_df.locationId.values))))
region_location=region_location[:20]
region_location

(('anwr', '31'),
 ('anwr', '32'),
 ('anwr', '33'),
 ('anwr', '34'),
 ('anwr', '35'),
 ('anwr', '36'),
 ('anwr', '37'),
 ('anwr', '38'),
 ('anwr', '39'),
 ('anwr', '40'),
 ('anwr', '41'),
 ('anwr', '42'),
 ('anwr', '43'),
 ('anwr', '44'),
 ('anwr', '45'),
 ('anwr', '46'),
 ('anwr', '47'),
 ('anwr', '48'),
 ('anwr', '49'),
 ('anwr', '50'))

In [18]:
clipping_results_path= "/home/enis/projects/nna/src/scripts/clipping_output/"
output_dir='/scratch/enis/data/nna/real/'

In [19]:

# with open(megan_labeled_files_info_path, 'r') as f:
#     lines = f.readlines()
#     lines = [i.strip().split(',') for i in lines]
region_location_datasets = {}
for region,location in region_location:
    
    filtered_files = file_properties_df[file_properties_df.region==region]
    filtered_files = filtered_files[filtered_files.locationId==location]
    dataset_name_v = "_".join([region,location])
    audio_dataset = dataimport.Dataset(dataset_name_v=dataset_name_v)
    for i in filtered_files.iterrows():
        audio_dataset[i[0]] = dataimport.Audio(i[1].name, float(i[1].durationSec))
    region_location_datasets[(region,location)]=audio_dataset



In [ ]:
for region,location in region_location:
    print(region,location)
    region_location_ins=region_location_datasets[(region,location)]
    region_location_ins.update_samples_w_clipping_info(output_folder=clipping_results_path)
#     print("Loading files...")
#     region_location_ins.load_audio_files()
#     region_location_ins.pick_channel_by_clipping()

    print("inference part")
    for audio_ins in region_location_ins.values():
        audio_ins.load_data()
        audio_ins.pick_channel_by_clipping(expected_len)
        input_file_data = repeat_data(audio_ins.data)

        # divide to 10 second excerpts
        input_file_data = input_file_data.reshape(-1,480000)
        input_file_data=torch.from_numpy(input_file_data).float()
        dataset = {'predict' : audioDatasetInfer(input_file_data, None, transform=transformCompose)}
        dataloader = {'predict': torch.utils.data.DataLoader(dataset['predict'], shuffle=False,batch_size=128)}

        outputs=[]
        for inputs, labels in dataloader['predict']:
            inputs = inputs.float().to(device)
            output = model_saved(inputs)
            output = output.to("cpu")
            index = output.data.numpy()
            outputs.append(index)
        outputs=np.concatenate(outputs)
        row = file_properties_df.loc[audio_ins.path]
        for i,n in [(0,'1-1-10'),(1,'1-1-7')]:
            sub_directory_addon='V1-'+n
            file_name_addon=sub_directory_addon
            file_name=fileUtils.standard_path_style(output_dir,row,sub_directory_addon=sub_directory_addon,file_name_addon=file_name_addon)
#             print(file_name)
            file_name.parent.mkdir(parents=True, exist_ok=True)
            np.save(str(file_name)+".npy", outputs[:,i])
        
        audio_ins.data = None



anwr 31
inference part


/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)


anwr 32
